In [2]:
import openai
from langchain.schema import HumanMessage 

In [20]:
def get_weather(city : str):
    if city == "Madrid":
        return 35
    elif city == "San Francisco":
        return 18
    elif city == "Paris":
        return 20
    else:
        return 15

In [21]:
import ollama

tools=[{
      'type': 'function',
      'function': {
        'name': 'get_weather',
        'description': 'Get the current weather for a city',
        'parameters': {
          'type': 'object',
          'properties': {
            'city': {
              'type': 'string',
              'description': 'The name of the city',
            },
          },
          'required': ['city'],
        },
      },
    },
  ]

In [22]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="llama3.1:8b",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what is the weather in Madrid?"},
  ],
    tools = tools
)
print(response.choices[0].message.content)

In [23]:
import openai
from langchain.schema import HumanMessage

def get_weather(city: str):
    if city == "Madrid":
        return 35
    elif city == "San Francisco":
        return 18
    elif city == "Paris":
        return 20
    else:
        return 15

# Mocking OpenAI client for demonstration purposes
class MockOpenAIClient:
    def __init__(self, base_url, api_key):
        self.base_url = base_url
        self.api_key = api_key

    def chat_completions_create(self, model, messages):
        user_message = messages[-1]['content']
        if "weather" in user_message:
            city = user_message.split()[-1].strip("?")
            temperature = get_weather(city)
            return {
                "choices": [
                    {
                        "message": {
                            "content": f"The current temperature in {city} is {temperature}°C."
                        }
                    }
                ]
            }

client = MockOpenAIClient(
    base_url='http://localhost:11434/v1',
    api_key='ollama',  # required, but unused
)

response = client.chat_completions_create(
    model="llama3.1:8b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "what is the weather in Madrid?"},
    ]
)

print(response["choices"][0]["message"]["content"])


The current temperature in Madrid is 35°C.


In [24]:
import openai

# Define the function to get the weather
def get_weather(city: str):
    weather_data = {
        "Madrid": 35,
        "San Francisco": 18,
        "Paris": 20,
    }
    return weather_data.get(city, 15)

# Function to handle the invocation of tools
def invoke_tool(tool_name, parameters):
    if tool_name == "get_weather":
        return get_weather(parameters['city'])
    else:
        raise ValueError(f"Tool {tool_name} not recognized.")

# Mock OpenAI client class to simulate API behavior
class MockOpenAIClient:
    def __init__(self, base_url, api_key):
        self.base_url = base_url
        self.api_key = api_key

    def chat_completions_create(self, model, messages, tools):
        user_message = messages[-1]['content']
        for tool in tools:
            tool_function = tool['function']
            if tool_function['name'] == 'get_weather' and "weather" in user_message:
                city = user_message.split()[-1].strip("?")
                response_content = invoke_tool(tool_function['name'], {'city': city})
                return {
                    "choices": [
                        {
                            "message": {
                                "content": f"The current temperature in {city} is {response_content}°C."
                            }
                        }
                    ]
                }

# Initialize the mock client
client = MockOpenAIClient(
    base_url='http://localhost:11434/v1',
    api_key='ollama',  # required, but unused
)

# Define the tools
tools = [{
    'type': 'function',
    'function': {
        'name': 'get_weather',
        'description': 'Get the current weather for a city',
        'parameters': {
            'type': 'object',
            'properties': {
                'city': {
                    'type': 'string',
                    'description': 'The name of the city',
                },
            },
            'required': ['city'],
        },
    },
}]

# Simulate the chat completion request
response = client.chat_completions_create(
    model="llama3.1:8b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the weather in Madrid?"},
    ],
    tools=tools
)

# Print the response
print(response["choices"][0]["message"]["content"])


The current temperature in Madrid is 35°C.


In [25]:
from autogen import AssistantAgent, UserProxyAgent

config_list = [
  {
    "model": "llama3.1:8b",
    "base_url": "http://localhost:11434/v1",
    "api_key": "ollama",
  }
]

assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})

user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False})
user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")

user_proxy (to assistant):

Plot a chart of NVDA and TESLA stock price change YTD.

--------------------------------------------------------------------------------
assistant (to user_proxy):

# filename: stock_chart.py
```python
import yfinance as yf
import matplotlib.pyplot as plt

def plot_stock_price_change(ticker, date):
    stock_data = yf.download(tickers=ticker, period=date)
    stock_data['Return'] = stock_data['Close'].pct_change()+1
    
    # Calculate the cumulative return
    stock_data['Cumulative Return'] = (stock_data['Return'] + 1)..cumprod() - 1
    
    plt.figure(figsize=(10,6))
    plt.plot(stock_data['Close'],label='Closing Price')
    plt.title('YTD Stock Performance: ' + ticker + '('+ date+')')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    
    return stock_data

nvda_data = plot_stock_price_change('NVDA', 'ytd')
print(nvda_data)

tesla_data = plot_stock_price_change('TSLA', 'ytd')
print(tesla_data)
```
Please save this code to a file named `stock_ch

Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  skip


user_proxy (to assistant):

skip

--------------------------------------------------------------------------------
assistant (to user_proxy):

There was no specific task that required me to collect information or perform a task using code, so we can just move on.

If you have any other questions or tasks you'd like to work on, feel free to ask! Otherwise, I'll be here if you need anything else. TERMINATE

--------------------------------------------------------------------------------


Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': 'Plot a chart of NVDA and TESLA stock price change YTD.', 'role': 'assistant'}, {'content': "# filename: stock_chart.py\n```python\nimport yfinance as yf\nimport matplotlib.pyplot as plt\n\ndef plot_stock_price_change(ticker, date):\n    stock_data = yf.download(tickers=ticker, period=date)\n    stock_data['Return'] = stock_data['Close'].pct_change()+1\n    \n    # Calculate the cumulative return\n    stock_data['Cumulative Return'] = (stock_data['Return'] + 1)..cumprod() - 1\n    \n    plt.figure(figsize=(10,6))\n    plt.plot(stock_data['Close'],label='Closing Price')\n    plt.title('YTD Stock Performance: ' + ticker + '('+ date+')')\n    plt.xlabel('Date')\n    plt.ylabel('Price (USD)')\n    \n    return stock_data\n\nnvda_data = plot_stock_price_change('NVDA', 'ytd')\nprint(nvda_data)\n\ntesla_data = plot_stock_price_change('TSLA', 'ytd')\nprint(tesla_data)\n```\nPlease save this code to a file named `stock_chart.py` and execute it.